In [1]:
####################################### PART ONE #######################################
import tensorflow as tf
import numpy as np
from PIL import Image
import datetime

NUM_CLASSES = 200
LIMIT = 40
NUM_CLASSES = LIMIT

IMAGES_MEAN = 122.5
IMAGES_STD = 63.32

LOGITS_COLLECTION = 'LOGITS'
LOGIT_LABELS_COLLECTION = 'LOGIT-LABELS'

RUN_PREFIX = datetime.datetime.fromtimestamp(datetime.datetime.now().timestamp()).strftime('%Y-%m-%d-%H_%M_%S')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    for name in image_names:
        label = int(name[:3])
        if label <= LIMIT:
            im = Image.open("images/" + name.rstrip('\n'))
            H, W = im.size
            pixels = list(im.getdata())
            if not type(pixels[0]) is int:
                # todo: right now we are discarding transparent images
                image = np.array([comp for pixel in pixels for comp in pixel]).reshape(-1, H, W, 3)
                images.append(image)
                # zero-index the label
                labels.append(label - 1)
        else: 
            break
    return images, labels

images_train, labels_train = load('train.txt')
images_test, labels_test = load('test.txt')

print(len(images_train))
print(len(images_test))

1199
1089


### NETWORK

In [3]:
BATCH_SIZE = 32
EPOCHS = 200
# OUT_CHANNEL_SIZES = [15, 30, 25, 18]

# todo: use tf.contrib.layers.conv2d for 3D filter
# def conv_relu(input_image, kernel_shape, bias_shape, stride = 2):
#     strides = [1, stride, stride, 1]
#     weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
#     biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
#     conv = tf.nn.conv2d(input_image, weights, strides=strides, padding='SAME')
#     return tf.nn.relu(conv + biases)

# def conv_layers(input_image, filter_height=5, filter_width=5, in_channels=3):
#     # Variables created here will be named "convX/weights", "convX/biases".
#     with tf.variable_scope("conv1"):
#         out_channels = OUT_CHANNEL_SIZES[0]
#         output = conv_relu(
#             input_image, 
#             kernel_shape=[filter_height, filter_width, in_channels, out_channels], 
#             bias_shape=[out_channels],
#             stride=1
#             )
# #     with tf.variable_scope("conv2"):
#         old_out_channels = out_channels
#         out_channels = OUT_CHANNEL_SIZES[1]
#         output = conv_relu(
#             output, 
#             kernel_shape=[filter_height, filter_width, old_out_channels, out_channels], 
#             bias_shape=[out_channels]
#             )
#     with tf.variable_scope("conv3"):
#         old_out_channels = out_channels
#         out_channels = OUT_CHANNEL_SIZES[2]
#         output = conv_relu(
#             output, 
#             kernel_shape=[filter_height, filter_width, old_out_channels, out_channels], 
#             bias_shape=[out_channels]
#             )
#     with tf.variable_scope("conv4"):
#         old_out_channels = out_channels
#         out_channels = OUT_CHANNEL_SIZES[3]
#         return conv_relu(
#             output, 
#             kernel_shape=[filter_height, filter_width, old_out_channels, out_channels], 
#             bias_shape=[out_channels]
#             )

# Modified from RikHeijdens on https://github.com/tensorflow/tensorflow/issues/6011
def spp_layer(image, dimensions=[6, 3, 2, 1]):
    # todo: fix this
    if tf.less(tf.shape(image)[1], dimensions[0] ** 2) is True:
        return None
    if tf.less(tf.shape(image)[2], dimensions[0] ** 2) is True:
        return None
    pool_list = []
    for pool_dim in dimensions:
        pool_list += max_pool_2d_nxn_regions(image, pool_dim)
    return tf.concat(pool_list, axis=1)

def max_pool_2d_nxn_regions(inputs, output_size):
    inputs_shape = tf.shape(inputs)
    h = tf.cast(tf.gather(inputs_shape, 1), tf.int32)
    w = tf.cast(tf.gather(inputs_shape, 2), tf.int32)

    result = []
    n = output_size
    for row in range(output_size):
        for col in range(output_size):
            # start_h = floor(row / n * h)
            start_h = tf.cast(tf.floor(tf.multiply(row / n, tf.cast(h, tf.float32))), tf.int32)
            # end_h = ceil((row + 1) / n * h)
            end_h = tf.cast(tf.ceil(tf.multiply((row + 1) / n, tf.cast(h, tf.float32))), tf.int32)
            # start_w = floor(col / n * w)
            start_w = tf.cast(tf.floor(tf.multiply(col / n, tf.cast(w, tf.float32))), tf.int32)
            # end_w = ceil((col + 1) / n * w)
            end_w = tf.cast(tf.ceil(tf.multiply((col + 1) / n, tf.cast(w, tf.float32))), tf.int32)
            pooling_region = inputs[:, start_h:end_h, start_w:end_w, :]
            pool_result = tf.reduce_max(pooling_region, axis=(1, 2))
            result.append(pool_result)
    return result


TEST_LOGITS_COLLECTION_BCONV = 'TEST_LOGITS_COLLECTION_BCONV'
TEST_LOGITS_COLLECTION = 'TEST_LOGITS_COLLECTION'

graph = tf.Graph()
with graph.as_default():

    image_placeholders = []
    label_placeholders = []

    with tf.variable_scope("network") as scope:
        conv_reuse = None
        for i in range(BATCH_SIZE):
            # todo: we can add transparent images 
            image = tf.placeholder(tf.float32, shape=(1,None,None,3), name='image_{}'.format(i))
            image_placeholders.append(image)
            label = tf.placeholder(tf.int64, shape=(), name='label_{}'.format(i))
            label_placeholders.append(label)

            logit = tf.to_float(image)
            logit = (logit - IMAGES_MEAN) / IMAGES_STD
#             logit = conv_layers(logit)
            tf.add_to_collection(TEST_LOGITS_COLLECTION_BCONV, tf.identity(logit, name='{}_{}'.format(TEST_LOGITS_COLLECTION_BCONV, i)))
            logit = tf.layers.conv2d(logit, 3, [2, 2], padding='SAME', reuse=conv_reuse, name='conv')
            conv_reuse = True
            tf.add_to_collection(TEST_LOGITS_COLLECTION, tf.identity(logit, name='{}_{}'.format(TEST_LOGITS_COLLECTION, i)))
            logit = spp_layer(logit)

            if not logit is None:
                logit = tf.reshape(logit, [-1])
                tf.add_to_collection(LOGITS_COLLECTION, tf.identity(logit, name='coll_logit_{}'.format(i)))
                tf.add_to_collection(LOGIT_LABELS_COLLECTION, tf.identity(label, name='coll_label_{}'.format(i)))

            scope.reuse_variables()
        
    logits_TEST_BCONV = tf.get_collection(TEST_LOGITS_COLLECTION_BCONV)
    logits_TEST = tf.get_collection(TEST_LOGITS_COLLECTION)
    
    logits = tf.stack(tf.get_collection(LOGITS_COLLECTION))
    logit_labels = tf.stack(tf.get_collection(LOGIT_LABELS_COLLECTION))
    print(logits.shape)

#     logits = tf.contrib.layers.fully_connected(logits, 650, activation_fn=tf.nn.relu, scope="fc1")
    logits = tf.contrib.layers.fully_connected(logits, 150, activation_fn=tf.nn.relu, scope="fc2")
    logits = tf.contrib.layers.fully_connected(logits, NUM_CLASSES, activation_fn=None, scope="fc3")

    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=logit_labels)) + 1e-6 * tf.losses.get_regularization_loss()
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        opt = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss)
    correct = tf.equal(tf.argmax(logits, -1), logit_labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    [print(v.name) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)]
    print('Number of trainable variables: {}'.format(len(tf.trainable_variables())))
    print('Total number of variables used {}'.format(np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()])))
    
sess = tf.Session(graph=graph)
LOG_DIR = 'log/{}'.format(RUN_PREFIX)
with graph.as_default(), sess.as_default():
    sess.run(tf.global_variables_initializer())
    print('ready to test')


(32, 150)
network/conv/kernel:0
network/conv/bias:0
fc2/weights:0
fc2/biases:0
fc3/weights:0
fc3/biases:0
network/conv/kernel/Momentum:0
network/conv/bias/Momentum:0
fc2/weights/Momentum:0
fc2/biases/Momentum:0
fc3/weights/Momentum:0
fc3/biases/Momentum:0
Number of trainable variables: 6
Total number of variables used 28729
ready to test


### TRAINING

In [ ]:
####################################### PART THREE #######################################
print('starting to test')
with graph.as_default(), sess.as_default():
#     writer = tf.summary.FileWriter(LOG_DIR)
#     writer.add_graph(sess.graph)
    for epoch in range(EPOCHS):
        np.random.seed(epoch)
        np.random.shuffle(images_train)
        np.random.seed(epoch)
        np.random.shuffle(labels_train)
        accuracy_vals, loss_vals = [], []
        for i in range(0, len(images_train) - BATCH_SIZE + 1, BATCH_SIZE):
            batch_images, batch_labels = images_train[i:i + BATCH_SIZE], labels_train[i:i + BATCH_SIZE]
        
            # todo: this is not very good... (probably replace with 1 x 1 x 1 x 1 when I implement SPP filter, do the same for training)
            if BATCH_SIZE - len(batch_images) > 0:
#                 print('testing diff: %d'%(BATCH_SIZE - len(batch_images)))
                for j in range(len(batch_images), BATCH_SIZE):
                    batch_images.append(images_train[j - len(batch_images)])
                    batch_labels.append(labels_train[j - len(batch_images)])

            fd = {**{k: v for k, v in zip(image_placeholders, batch_images)}, **{k: v for k, v in zip(label_placeholders, batch_labels )}}

            accuracy_val, loss_val, _ = sess.run([accuracy, loss, opt], feed_dict=fd)
            accuracy_vals.append(accuracy_val)
            loss_vals.append(loss_val)
            
#             if i >= BATCH_SIZE * 3:
#                 o_logits, o_logits_TEST, o_logits_TEST_BCONV = sess.run([logits, logits_TEST, logits_TEST_BCONV], feed_dict=fd)
#                 print('[{}:{}] LOGIT LEN: {} ACC: {} LOSS: {}'.format(epoch, i, len(o_logits), accuracy_val, loss_val))
#                 print(o_logits_TEST_BCONV[0])
#                 print('[{}:{}] ==========================='.format(epoch, i))
#                 print(o_logits_TEST[0])
        val_correct = []
        for i in range(0, len(images_test), BATCH_SIZE):
            batch_images, batch_labels = images_test[i:i + BATCH_SIZE], labels_test[i:i + BATCH_SIZE]
            
            if BATCH_SIZE - len(batch_images) > 0:
#                 print('training diff: %d'%(BATCH_SIZE - len(batch_images)))
                for j in range(len(batch_images), BATCH_SIZE):
                    batch_images.append(images_test[j - len(batch_images)])
                    batch_labels.append(labels_test[j - len(batch_images)])
                
            fd = {**{k: v for k, v in zip(image_placeholders, batch_images)}, **{k: v for k, v in zip(label_placeholders, batch_labels )}}
            c = sess.run(correct, feed_dict=fd)
            val_correct.extend(c)

        # s = tf.Summary()
        # s.value.add(simple_value=np.mean(accuracy_vals), tag="accuracy")
        # s.value.add(simple_value=np.mean(loss_vals), tag="loss")
        # s.value.add(simple_value=np.mean(val_correct), tag="correct")
        # writer.add_summary(s, epoch)
        # writer.flush()
        # saver.save(sess, path.join(RUN_PREFIX, 'network.ckpt'), global_step=epoch)
        print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f  \t  validation accuracy: %0.3f'%(epoch, np.mean(accuracy_vals), np.mean(loss_vals), np.mean(val_correct)))


# sess.run([accuracy], feed_dict={image: images_train[0]})
# batch_images = images_train[:BATCH_SIZE]
# batch_labels = labels_train[:BATCH_SIZE]
# fd = {**{i: d for i, d in zip(image_placeholders, batch_images)}, **{i: d for i, d in zip(label_placeholders, batch_labels )}}
# accuracy_val, loss_val, _  = sess.run([accuracy, total_loss, opt], feed_dict=fd)
# print(accuracy_val)
# print(loss_val)
# print(logits[0])

# batch_images = images_train[BATCH_SIZE:BATCH_SIZE+BATCH_SIZE]
# batch_labels = labels_train[BATCH_SIZE:BATCH_SIZE+BATCH_SIZE]
# accuracy_val, loss_val, _  = session.run([accuracy, total_loss, opt], feed_dict=fd)
# print(accuracy_val)
# print(loss_val)

# print(logits[0])

starting to test
[  0] Accuracy: 0.071  	  Loss: 3.598  	  validation accuracy: 0.063
[  1] Accuracy: 0.076  	  Loss: 3.573  	  validation accuracy: 0.078
[  2] Accuracy: 0.097  	  Loss: 3.543  	  validation accuracy: 0.092
[  3] Accuracy: 0.100  	  Loss: 3.511  	  validation accuracy: 0.088
[  4] Accuracy: 0.111  	  Loss: 3.475  	  validation accuracy: 0.103
[  5] Accuracy: 0.108  	  Loss: 3.431  	  validation accuracy: 0.113
[  6] Accuracy: 0.120  	  Loss: 3.391  	  validation accuracy: 0.117
[  7] Accuracy: 0.126  	  Loss: 3.347  	  validation accuracy: 0.111
[  8] Accuracy: 0.136  	  Loss: 3.304  	  validation accuracy: 0.122
[  9] Accuracy: 0.142  	  Loss: 3.267  	  validation accuracy: 0.124
[ 10] Accuracy: 0.143  	  Loss: 3.222  	  validation accuracy: 0.132
[ 11] Accuracy: 0.157  	  Loss: 3.174  	  validation accuracy: 0.146
[ 12] Accuracy: 0.153  	  Loss: 3.137  	  validation accuracy: 0.144
[ 13] Accuracy: 0.164  	  Loss: 3.098  	  validation accuracy: 0.152
[ 14] Accuracy: 0